<a href="https://colab.research.google.com/github/dapoarr/Materi-Mechine-Learning/blob/main/Latihan_Multivariate_Time_Series_BPML_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf

In [ ]:
# https://drive.google.com/file/d/1AZRfFoyekqSYpri5183RmJjciRGz_ood/view?usp=sharing
df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)
df

In [ ]:
N_FEATURES = len(df.columns)

In [ ]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

In [ ]:
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

In [ ]:
# Splits the data into training and validation sets.
SPLIT_TIME = int(len(data) * 0.5) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

In [ ]:
len(data)

86400

In [ ]:
SPLIT_TIME

43200

In [ ]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1

In [ ]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
# Code to create windowed train and validation datasets.
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

In [ ]:
# model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(N_FEATURES)
#     ])

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(72, activation='relu', input_shape=(N_PAST, N_FEATURES), return_sequences=True),
        tf.keras.layers.LSTM(48, activation='relu', return_sequences=True),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(N_FEATURES)
    ])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('mae') < 0.055 and logs.get('val_mae') < 0.055):
                self.model.stop_training = True

callbacks = myCallback()

In [ ]:
# Code to train and compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=tf.keras.losses.Huber(),
                  optimizer= optimizer,
                  metrics=["mae"])

In [ ]:
model.fit(train_set,
          validation_data=(valid_set),
          epochs=100,
          callbacks=callbacks
    )

Epoch 1/100
   1348/Unknown 74s 46ms/step - loss: 0.0152 - mae: 0.1022

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 94s 60ms/step - loss: 0.0152 - mae: 0.1022 - val_loss: 0.0105 - val_mae: 0.0767
Epoch 2/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 67s 50ms/step - loss: 0.0108 - mae: 0.0817 - val_loss: 0.0102 - val_mae: 0.0748
Epoch 3/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 84s 51ms/step - loss: 0.0103 - mae: 0.0788 - val_loss: 0.0098 - val_mae: 0.0716
Epoch 4/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 66s 49ms/step - loss: 0.0100 - mae: 0.0769 - val_loss: 0.0098 - val_mae: 0.0717
Epoch 5/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 83s 50ms/step - loss: 0.0099 - mae: 0.0754 - val_loss: 0.0103 - val_mae: 0.0756
Epoch 6/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 67s 49ms/step - loss: 0.0098 - mae: 0.0748 - val_loss: 0.0095 - val_mae: 0.0702
Epoch 7/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 141s 93ms/step - loss: 0.0097 - mae: 0.0739 - val_loss: 0.0095 - val_mae: 0.0718
Epoch 8/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 83s 49ms/step - loss: 0.0096 - mae: 0.0735 - val_loss: 0.0095 - val_mae: 0.0717
Epoch 9/100
1349/1349 ━━━━━

In [ ]:
train_pred = model.predict(train_set)

In [ ]:
train_pred[0][0]

array([0.31021965, 0.2034628 , 0.04024957, 0.3028689 , 0.03431462,
       0.00725988, 0.89442176], dtype=float32)